In [1]:
#import libraries
import pandas as pd
import json
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import warnings
import pprint
warnings.simplefilter('ignore')

In [2]:
url ='https://api.spoonacular.com/recipes/complexSearch?apiKey&cuisine=American&query=brunch&addRecipeInformation=True&addRecipeNutrition=True&number=100'
new_res4= requests.get(url).json()
#new_res4 

In [3]:
recipe8_df=pd.json_normalize(new_res4,record_path=['results'])
recipe8_df.head()

,vegetarian,vegan,glutenFree,dairyFree,veryHealthy,cheap,veryPopular,sustainable,weightWatcherSmartPoints,gaps,...,nutrition.flavonoids,nutrition.ingredients,nutrition.caloricBreakdown.percentProtein,nutrition.caloricBreakdown.percentFat,nutrition.caloricBreakdown.percentCarbs,nutrition.weightPerServing.amount,nutrition.weightPerServing.unit,preparationMinutes,cookingMinutes,author
0,False,False,True,False,False,False,False,False,32,no,...,"[{'name': 'Cyanidin', 'amount': 0.33, 'unit': ...","[{'id': 9037, 'name': 'avocado', 'amount': 0.5...",20.92,72.24,6.84,497,g,NaN,NaN,NaN
1,True,False,True,True,False,False,False,False,7,no,...,"[{'name': 'Cyanidin', 'amount': 0.0, 'unit': '...","[{'id': 4047, 'name': 'coconut oil', 'amount':...",6.77,30.44,62.79,240,g,NaN,NaN,NaN
2,True,False,False,False,False,False,False,False,10,no,...,"[{'name': 'Cyanidin', 'amount': 6.67, 'unit': ...","[{'id': 18371, 'name': 'baking powder', 'amoun...",5.87,30.45,63.68,149,g,20.0,30.0,NaN
3,False,False,False,True,False,False,False,False,23,no,...,"[{'name': 'Cyanidin', 'amount': 3.82, 'unit': ...","[{'id': 93607, 'name': 'almond milk', 'amount'...",7.94,37.70,54.36,408,g,NaN,NaN,NaN
4,False,False,False,True,False,False,False,False,5,no,...,"[{'name': 'Cyanidin', 'amount': 0.0, 'unit': '...","[{'id': 93801, 'name': 'hollandaise sauce', 'a...",28.49,41.68,29.83,110,g,NaN,NaN,NaN


In [4]:
recipe8_df.drop(columns=['vegetarian', 'vegan', 'glutenFree', 'dairyFree', 'veryHealthy', 'cheap', 'veryPopular', 'sustainable', 'weightWatcherSmartPoints', 'gaps', 'lowFodmap', 'aggregateLikes', 'spoonacularScore',
                      'healthScore', 'creditsText', 'license', 'sourceName', 'pricePerServing','sourceUrl','imageType','summary','diets', 'occasions','spoonacularSourceUrl','nutrition.properties','nutrition.flavonoids','nutrition.caloricBreakdown.percentProtein','nutrition.caloricBreakdown.percentFat','nutrition.caloricBreakdown.percentCarbs','nutrition.weightPerServing.amount',
                      'nutrition.weightPerServing.unit','preparationMinutes','cookingMinutes','author','nutrition.nutrients','nutrition.ingredients'],inplace=True)
#recipe8_df

# One hot encoding dishType column

In [5]:
recipe8_df['dish']=0
recipe8_df.columns

Index(['id', 'title', 'readyInMinutes', 'servings', 'image', 'cuisines',
       'dishTypes', 'analyzedInstructions', 'dish'],
      dtype='object')

In [6]:
for i in range(0,12):
    item =recipe8_df['dishTypes'][i]
    if len(item)==0:
        recipe8_df.iloc[i,8]=0
        
    else:
        #print(item[0])
        
        #if any(x in ['lunch', 'dinner', 'antipasti','side dish','salad'] for x in item):
            #print(f"{i}:lunch")
            #recipe5_df.iloc[i,7]=1
        if any(x in ['breakfast', 'bread'] for x in item):
            recipe8_df.iloc[i,8]=0
#recipe8_df

In [7]:
recipe8_df.drop(columns=['dishTypes'],inplace=True)
recipe8_df.head(3)

,id,title,readyInMinutes,servings,image,cuisines,analyzedInstructions,dish
0,650377,Low Carb Brunch Burger,30,2,https://spoonacular.com/recipeImages/650377-31...,[American],"[{'name': '', 'steps': [{'number': 1, 'step': ...",0
1,662584,"Sweet Potato Kimchi Hash Brunch - gluten free,...",45,6,https://spoonacular.com/recipeImages/662584-31...,"[Korean, Asian]","[{'name': '', 'steps': [{'number': 1, 'step': ...",0
2,729530,Upside-Down Blueberry Puffs Brunch,50,6,https://spoonacular.com/recipeImages/729530-31...,[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",0


In [8]:
#Getting ingredients for each recipe
#Ingredients is a list of dictionaries in nutrition key
ingredients_list=[]
recipe_idlist=[]

for recipe in new_res4['results']:
    recipe_id =recipe['id']
    ingres=recipe['nutrition']['ingredients']
    ingri_dict={}
    for ingredient_dict in ingres:
        name=ingredient_dict['name']
        amount=ingredient_dict['amount']
        unit=ingredient_dict['unit']
        value=str(round(amount,2))+' '+unit 
        ingri_dict[name]=value
    ingredients_list.append(ingri_dict)
    recipe_idlist.append(recipe_id)  

In [9]:
ingredient_names=[]
for i in range(0,12):
    ingredients=list(ingredients_list[i])
    ingredient_names.append(ingredients)
len(ingredient_names)

12

In [10]:
recipe8_df['ingredients']=ingredients_list

In [11]:
recipe8_df['grocery']=ingredient_names
recipe8_df.head(3)

,id,title,readyInMinutes,servings,image,cuisines,analyzedInstructions,dish,ingredients,grocery
0,650377,Low Carb Brunch Burger,30,2,https://spoonacular.com/recipeImages/650377-31...,[American],"[{'name': '', 'steps': [{'number': 1, 'step': ...",0,"{'avocado': '0.5 ', 'bacon': '2.0 strips', 'cu...","[avocado, bacon, cumin, eggs, garlic powder, g..."
1,662584,"Sweet Potato Kimchi Hash Brunch - gluten free,...",45,6,https://spoonacular.com/recipeImages/662584-31...,"[Korean, Asian]","[{'name': '', 'steps': [{'number': 1, 'step': ...",0,"{'coconut oil': '0.5 tablespoons', 'onions': '...","[coconut oil, onions, garlic, sweet potatoes, ..."
2,729530,Upside-Down Blueberry Puffs Brunch,50,6,https://spoonacular.com/recipeImages/729530-31...,[],"[{'name': '', 'steps': [{'number': 1, 'step': ...",0,"{'baking powder': '0.25 teaspoons', 'blueberri...","[baking powder, blueberries, butter, cream of ..."


# EXtracting Instructions

In [12]:
recipe_instructions=recipe8_df[['analyzedInstructions','id']]
#recipe_instructions.head

In [13]:
recipe_inst=[]
for instructions,ids in zip(list(recipe_instructions['analyzedInstructions'].values),list(recipe_instructions['id'].values)):
    #print(instructions,ids)
    #print(instructions[0]['steps'])
    instruct={}
    recipe_step=''
    try:
    #break   
        for no_steps in range(len(instructions[0]['steps'])):
            #print(no_steps)
            recipe_step+=instructions[0]['steps'][no_steps]['step']    
            #break
            #print(recipe_step)
        instruct[ids]=recipe_step
        #print(instruct)
        recipe_inst.append(instruct)
    except:
        pass

In [14]:
instruction_final=pd.DataFrame()
for dict_recipe in recipe_inst:
    df=pd.DataFrame(dict_recipe,index=dict_recipe.keys())
    df.columns=['instructions']
    #print(df)
    #break
    instruction_final=pd.concat([instruction_final,df],ignore_index=False,axis=0)

In [15]:
instruction_final.index.names = ['id']
#instruction_final

In [16]:
recipe8_df=recipe8_df.merge(instruction_final,on='id')
recipe8_df.drop(columns=['analyzedInstructions'],inplace=True)

In [17]:
recipe8_df.head(3)

,id,title,readyInMinutes,servings,image,cuisines,dish,ingredients,grocery,instructions
0,650377,Low Carb Brunch Burger,30,2,https://spoonacular.com/recipeImages/650377-31...,[American],0,"{'avocado': '0.5 ', 'bacon': '2.0 strips', 'cu...","[avocado, bacon, cumin, eggs, garlic powder, g...",Fry up 4 strips of bacon and set aside.Leave s...
1,662584,"Sweet Potato Kimchi Hash Brunch - gluten free,...",45,6,https://spoonacular.com/recipeImages/662584-31...,"[Korean, Asian]",0,"{'coconut oil': '0.5 tablespoons', 'onions': '...","[coconut oil, onions, garlic, sweet potatoes, ...",Set a large skillet over medium high heat.When...
2,729530,Upside-Down Blueberry Puffs Brunch,50,6,https://spoonacular.com/recipeImages/729530-31...,[],0,"{'baking powder': '0.25 teaspoons', 'blueberri...","[baking powder, blueberries, butter, cream of ...",Preheat oven to 375 degrees.Wash blueberries a...


In [18]:
#Saving 100 recipes dataframe to a csv
recipe8_df.to_csv('./all_recipes/recipe8.csv',index=False)

# Extracting nutrition data

In [19]:
nutri_list =[]
id_list=[]
for recipe in new_res4['results']:
    recipe_id =recipe['id']
    nutrients =recipe['nutrition']['nutrients']
    nutri_dict={}
    for nutrient_dict in nutrients:
        name = nutrient_dict['name']
        amount = nutrient_dict['amount']
        unit = nutrient_dict['unit']
        #value = str(round(amount,2))+' '+unit
        key=name+' '+unit
        #print(name)
        nutri_dict[key]=amount
    nutri_list.append(nutri_dict)
    id_list.append(recipe_id)

In [20]:
nutrition8_df= pd.DataFrame(nutri_list,index=id_list)
nutrition8_df.index.name='recipe_id'
nutrition8_df.head(3)

,Calories kcal,Fat g,Saturated Fat g,Carbohydrates g,Net Carbohydrates g,Sugar g,Cholesterol mg,Sodium mg,Protein g,Vitamin C mg,...,Fiber g,Vitamin E mg,Vitamin K µg,Vitamin B1 mg,Magnesium mg,Copper mg,Manganese mg,Calcium mg,Vitamin D µg,Alcohol g
recipe_id,,,,,,,,,,,,,,,,,,,,,
650377,1028.27,82.55,27.02,17.57,8.74,4.22,354.80,732.38,53.80,105.74,...,8.83,4.90,29.12,0.37,94.72,0.43,0.34,108.87,1.29,NaN
662584,223.10,7.78,6.06,36.12,30.82,9.11,27.28,870.66,3.89,7.87,...,5.30,0.51,6.21,0.14,43.92,0.26,0.52,64.74,NaN,NaN
729530,276.14,9.62,5.61,45.24,42.77,25.26,49.87,185.98,4.17,7.65,...,2.48,0.78,16.04,0.20,12.04,0.09,0.42,73.21,0.47,NaN


In [21]:
#saving the nutrition details of 100 recipes dataframe to csv
nutrition8_df.to_csv('./all_nutrition/nutrition8.csv',index=True)

In [22]:
#Defining dataframe for ingredients
ingres8_df=pd.DataFrame(ingredients_list,index= recipe_idlist)
ingres8_df.fillna(value=0,inplace=True)
ingres8_df.index.name='recipe_id'
ingres8_df.head(3)

,avocado,bacon,cumin,eggs,garlic powder,ground beef,onion powder,bell pepper,salt,sharp cheddar,...,olive oil,mushrooms,asparagus,garlic clove,italian seasonings,tumeric,cayenne pepper,water,fries,red potatoes
recipe_id,,,,,,,,,,,,,,,,,,,,,
650377,0.5,2.0 strips,1.0 servings,1.0,1.0 servings,0.5 lb,1.0 servings,1.0 servings,1.0 servings,1.0 large slices,...,0,0,0,0,0,0,0,0,0,0
662584,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
729530,0,0,0,0,0,0,0,0,0.04 teaspoon,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
#saving the ingredients of 100 recipes dataframe to csv
ingres8_df.to_csv('./all_ingredients/ingredients8.csv',index=True)